In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import sys
import zipfile

from natsort import natsorted
from dotenv import find_dotenv


sys.path.append(os.path.dirname(find_dotenv()))

In [ ]:
from src import DMD
from src import snapshots_assembly

Before downloading dataset, you need to create an account on Kaggle. Instructions for using the Kaggle API can be seen at https://www.kaggle.com/docs/api
This dataset contains 991MB and two versions of the simulation covid spread lombardy etc

In [ ]:
FORCE_DOWNLOAD = False
files_core = "covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/"
if not os.path.exists(files_core) or FORCE_DOWNLOAD: 
    ! kaggle datasets download -d gfbarros/covid-19-spread-in-lombardy-italy

"Zip file corrupted or not a Zip file. Check zip file location or integrity"
"(or try downloading dataset again by switch FORCE_DOWNLOAD"
"to True on the previous Jupyter Notebook cell)."

We need to unzip etc etc etc
points to data folder

In [ ]:
def unpack_kaggle_dataset(filepath):
    with zipfile.ZipFile("covid-19-spread-in-lombardy-italy.zip", "r") as zip_ref:
        zip_ref.extractall(".")
    return


unpack_kaggle_dataset("covid-19-spread-in-lombardy-italy.zip")

Let's instantiate the dataset etc etc etc

In [19]:
files_core = "covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/"
os_walk_files = next(os.walk(files_core))
folders = natsorted(os_walk_files[1])

filenames = [
    Path(files_core) / Path(snapshot_folder) / Path(f"out_1_000_{str(i).zfill(5)}.h5")
    for i, snapshot_folder in enumerate(folders)
]
# for root, directory, filename in :
#     print(root, filename)
#     filenames.append(Path(files_core) / Path(directory) / Path(filename))

In [20]:
filenames[:5]

[PosixPath('covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step0/out_1_000_00000.h5'),
 PosixPath('covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step1/out_1_000_00001.h5'),
 PosixPath('covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step2/out_1_000_00002.h5'),
 PosixPath('covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step3/out_1_000_00003.h5'),
 PosixPath('covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step4/out_1_000_00004.h5')]

In [21]:
snapshot_ingestion_parameters = {
    "filenames": filenames,
    "dataset": "s",
}

In [22]:
dataset = snapshots_assembly(
    "h5_libmesh", snapshot_ingestion_parameters=snapshot_ingestion_parameters
)

2023-04-15 18:21:43,321 - src.preprocessing.build_snapshots - INFO - Starting choice of file type:
2023-04-15 18:21:43,323 - src.preprocessing.build_snapshots - INFO - libMesh/EdgeCFD HDF5 file selected.


In [23]:
dataset.shape

(13158, 482)

In [24]:
files_core = "covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/"
os_walk_files = next(os.walk(files_core))
files = natsorted(os_walk_files[2])

filenames = [
    Path(files_core) / Path(f"{str(i)}infected.vtk")
    for i, _ in zip(range(4, 480, 4), files)
]
# for root, directory, filename in :
#     print(root, filename)
#     filenames.append(Path(files_core) / Path(directory) / Path(filename))

In [25]:
filenames[:5]

[PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/4infected.vtk'),
 PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/8infected.vtk'),
 PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/12infected.vtk'),
 PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/16infected.vtk'),
 PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/20infected.vtk')]

In [26]:
filenames[-5:]

[PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/460infected.vtk'),
 PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/464infected.vtk'),
 PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/468infected.vtk'),
 PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/472infected.vtk'),
 PosixPath('covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/476infected.vtk')]

In [27]:
snapshot_ingestion_parameters = {
    "filenames": filenames,
    "starting_line": 125939,
    "ending_line": 210239,
}

In [28]:
dataset = snapshots_assembly(
    "vtk_freefem", snapshot_ingestion_parameters=snapshot_ingestion_parameters
)

2023-04-15 18:21:53,778 - src.preprocessing.build_snapshots - INFO - Starting choice of file type:
2023-04-15 18:21:53,779 - src.preprocessing.build_snapshots - INFO - FreeFem++ vtk file selected.


In [29]:
dmd_parameters = {
    "factorization_algorithm": "randomized_svd",
    "basis_vectors": 50,
    "randomized_svd_parameters": {
        "power_iterations": 1,
        "oversampling": 20,
    },
    "starting_step": 20,
    "dt_simulation": 0.05,
}

In [30]:
dmd = DMD(dataset, dmd_parameters)

In [ ]:
dmd.factorization()

In [ ]:
dmd.dmd_approximation["s"]

In [ ]:
dmd.dmd_core()

In [ ]:
import numpy as np


def compute_frobenius_norm(mat_a, mat_b):
    return np.linalg.norm(mat_a - mat_b)

In [ ]:
compute_frobenius_norm(dmd.snapshots_matrix, dmd.dmd_approximation["dmd_matrix"])

In [ ]:
dmd.dmd_approximation.keys()